In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import random
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import csv
from sklearn.cluster import DBSCAN
import math
import statistics
pd.options.mode.chained_assignment = None  # default='warn'
import itertools

#Convert CSV to Dataframe
data = pd.read_csv("C:/Users/Siloux/Downloads/pairwise_user_experiment (6).csv", header=[0])
data["FOLDER"] = data["IN FILE"].apply(lambda x: "")
data["LAST MOD BY"] = data["LAST MOD BY"].apply(lambda x: x.split(" ")[0])
data

,IN FILE,CLIP LENGTH,OFFSET,DURATION,MAX FREQ,MIN FREQ,SAMPLE RATE,MANUAL ID,TIME_SPENT,LAST MOD BY,CONFIDENCE,RETIRED,COUNTED,FOLDER
0,XC23899_-_White-vented_Shama_-_Copsychus_niger...,61.1034,1.1839,1.3399,24000,0,48000,White-vented Shama,141.864,spres,0.577908,False,0,
1,XC23899_-_White-vented_Shama_-_Copsychus_niger...,61.1034,6.9715,1.5479,24000,0,48000,White-vented Shama,141.864,spres,0.577908,False,0,
2,XC23899_-_White-vented_Shama_-_Copsychus_niger...,61.1034,8.8594,0.4500,24000,0,48000,White-vented Shama,141.864,spres,0.577908,False,0,
3,XC23899_-_White-vented_Shama_-_Copsychus_niger...,61.1034,9.9393,1.0800,24000,0,48000,White-vented Shama,141.864,spres,0.577908,False,0,
4,XC23899_-_White-vented_Shama_-_Copsychus_niger...,61.1034,15.1890,0.9299,24000,0,48000,White-vented Shama,141.864,spres,0.577908,False,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2243,XC688950_-_Balearic_Warbler_-_Sylvia_balearica...,25.9531,22.3916,0.8100,24000,0,48000,bird,311.194,Anshul,0.300511,False,0,
2244,XC688950_-_Balearic_Warbler_-_Sylvia_balearica...,25.9531,23.7316,0.1600,24000,0,48000,bird,311.194,Anshul,0.300511,False,0,
2245,XC688950_-_Balearic_Warbler_-_Sylvia_balearica...,25.9531,24.2615,0.1200,24000,0,48000,bird,311.194,Anshul,0.300511,False,0,
2246,XC688950_-_Balearic_Warbler_-_Sylvia_balearica...,25.9531,24.7815,0.1600,24000,0,48000,bird,311.194,Anshul,0.300511,False,0,


#### Import Confidence Functions and Create List of Functions

In [3]:
from confidence_functions.confidence import *
 
confidence_funcs = [ get_silhoutte_confidence, get_silhoutte_users_confidence, majority_vote, get_pairwise_iou]
confidence_funcs

[<function confidence_functions.confidence.get_silhoutte_confidence(df, users)>,
 <function confidence_functions.confidence.get_silhoutte_users_confidence(df, users)>,
 <function confidence_functions.confidence.majority_vote(df, users, chunk_length=1)>,
 <function confidence_functions.confidence.get_pairwise_iou(df, users)>]

### permuatation code

In [4]:
def permutate_users(df, num_users, clip_name, confidence_func):
    #Get data for indivual clip
    results = []
    df = df[df["IN FILE"] == clip_name]
    users = df["LAST MOD BY"].unique()
    
    for user_permutation in itertools.permutations(users, num_users):
        try:
            results.append(confidence_func(df, user_permutation))
        except:
            pass
        
    results = np.mean(results)
    return results
    
#permutate_users(data, 2, "XC425555_-_Screaming_Piha_-_Lipaugus_vociferans.mp3", get_silhoutte_users_confidence)

### Clip Spefific Experiment

In [5]:
def determine_confidence_by_clip(df, clip_name, confidence_funcs):
    results = {}
    df = df[df["IN FILE"] == clip_name]
    users = df["LAST MOD BY"].unique()
    for confidence_func in confidence_funcs:
        print("working with function: " + confidence_func.__name__)
        results[confidence_func] = {}
        
        for user_count in range(6):
            results[confidence_func][user_count+1]= permutate_users(df, user_count+1, clip_name, confidence_func)
    return results

def determine_confidence_by_func(df, confidence_funcs):
    results = {}
    users = df["LAST MOD BY"].unique()
    files = df["IN FILE"].unique()
    for func in confidence_funcs:
        results[func] = {}
        for user_count in range(6):
            print("USER", user_count)
            confidence = np.array([])
            for file in files:
                if (user_count < len(df[df["IN FILE"] == file]["LAST MOD BY"].unique())):
                       confidence = np.append(confidence, permutate_users(df, user_count+1, file, func))
            print(confidence)
            confidence = confidence.mean()
            results[func][user_count+1] = confidence
    return results
        
#determine_confidence_by_clip(data, "XC425555_-_Screaming_Piha_-_Lipaugus_vociferans.mp3", confidence_funcs)
#determine_confidence_by_func(data, confidence_funcs)

# Full Experiment

In [6]:
def run_experiment(df, confidence_funcs):
    #Get the adverage confidence over number of users for each file
    results = {"by_clip": {}, "Over_all_clips": {}}
    filenames = df["IN FILE"].unique()
    #for file in filenames:
    #    print("working with file: " + file)
    #    results["by_clip"][file] = determine_confidence_by_clip(df, file, confidence_funcs)
    
    print("Get adverage over all files")
    results["Over_all_clips"] = determine_confidence_by_func(df, confidence_funcs)
        
        
    
    return results

#experiment_results = run_experiment(data, confidence_funcs)
#experiment_results

In [7]:
def generate_graph(experiment_results):
    for filename in experiment_results:
        #For each filename, plot the performance of each confidence function
        for confidence_func in experiment_results[filename]:
            confidence_dirc = experiment_results[filename][confidence_func]
            users_count = list(confidence_dirc.keys())
            score = list(confidence_dirc.values())
            plt.plot(users_count, score, label=(confidence_func.__name__))
        
        #Throw in some fancy formatting
        plt.legend(loc='best')
        plt.title(filename)
        plt.xlabel("User Count")
        plt.ylabel("Average Confidence of Function")
        plt.show()

#generate_graph(experiment_results["by_clip"])

In [8]:
def generate_graph_all(experiment_results):
    #For each filename, plot the performance of each confidence function
    for confidence_func in experiment_results:
        confidence_dirc = experiment_results[confidence_func]
        users_count = list(confidence_dirc.keys())
        score = list(confidence_dirc.values())
        plt.plot(users_count, score, label=(confidence_func.__name__))

    #Throw in some fancy formatting
    plt.legend(loc='best')
    #plt.title(filename)
    plt.xlabel("User Count")
    plt.ylabel("Average Confidence of Function")
    plt.show()
    
    
    
    
#generate_graph_all(experiment_results["Over_all_clips"])

## Experiment Design 2

In [14]:
def experiment_via_random_orders(df, confidence_funcs, num_of_orders=1000):
    results = {}
    users = df["LAST MOD BY"].unique()
    files = df["IN FILE"].unique()
    for func in confidence_funcs:
        results[func.__name__] = {}
        for i in range (len(users)):
            results[func.__name__][i] = []
    
    
    for file in files:
        print("on ", file, "of", len(files))
        tmp_df = df[df["IN FILE"] == file]
        users = tmp_df["LAST MOD BY"].unique()

        to_be_randomly_inserted = np.tile(users, (num_of_orders, 1))
        to_be_randomly_inserted = pd.DataFrame(to_be_randomly_inserted)
        to_be_randomly_inserted = to_be_randomly_inserted.apply(lambda x:np.random.shuffle(x) or x, axis=1)
        random_insertion_orders = to_be_randomly_inserted.values
        
        count = 0
        for insertion_order in random_insertion_orders:
            print(count)
            count += 1
            for func in confidence_funcs: 
                for user_count in range(1,len(insertion_order)):
                    curr_insert = insertion_order[:user_count+1]
                    tmp_df_users = tmp_df[tmp_df["LAST MOD BY"].isin(curr_insert)]
                    try:
                        results[func.__name__][user_count].append(func(tmp_df_users, curr_insert))
                    except:
                        pass
    return results

experiment_results = experiment_via_random_orders(data, confidence_funcs, 1000)
experiment_results

on  XC23899_-_White-vented_Shama_-_Copsychus_niger.mp3 of 25


C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\pandas\core\apply.py:828: UserWarning: `x` isn't a recognized object; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  results[i] = self.f(v)


0
on  XC163810_-_Silvery-flanked_Antwren_-_Myrmotherula_luctuosa.mp3 of 25
0


C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\pandas\core\apply.py:828: UserWarning: `x` isn't a recognized object; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  results[i] = self.f(v)


on  XC235919_-_Baglafecht_Weaver_-_Ploceus_baglafecht.mp3 of 25
0


...

on  XC688945_-_Balearic_Warbler_-_Sylvia_balearica.mp3 of 25
0


C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\pandas\core\apply.py:828: UserWarning: `x` isn't a recognized object; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  results[i] = self.f(v)


on  XC688950_-_Balearic_Warbler_-_Sylvia_balearica.mp3 of 25
0


C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\pandas\core\apply.py:828: UserWarning: `x` isn't a recognized object; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  results[i] = self.f(v)


{'get_silhoutte_confidence': {0: [],
  1: [0.9228374114200125,
   0.9916753096104576,
   0.444912868876559,
   0.5286072926118901,
   0.8399023063591474,
   0.8101631755515453,
   0,
   0,
   0.8630633612693448,
   0.9880545489757687,
   0.5224287125873076,
   0.7655584576672219,
   0.5587578868621627,
   0.9108491111434109,
   0.9802632497017835,
   0.9640227004704329,
   0.8488286062765426,
   0.8524379571538377,
   0.6637629012824463,
   0,
   0.9197308046554844,
   0.8740212212991818,
   0,
   0.8272047810094683,
   0.2759910921529042],
  2: [0.9536987917506862,
   0.8989509003387748,
   0.5481885916848738,
   0.37164183316101573,
   0.9253671474558207,
   0.8771320891469716,
   0.31046466994554267,
   0,
   0.8801075854199869,
   0.9871960976815842,
   0.48463693002858454,
   0.725053787291138,
   0.7103007904303162,
   0.9701708589715807,
   0.9811146491025737,
   0.9548879276595852,
   0.8630605472830313,
   0.7576597278263456,
   0.6201428731352271,
   0.848186918545766,
   0.9

In [15]:
#generate_graph_all(experiment_results)

In [16]:
experiment_results

{'get_silhoutte_confidence': {0: [],
  1: [0.9228374114200125,
   0.9916753096104576,
   0.444912868876559,
   0.5286072926118901,
   0.8399023063591474,
   0.8101631755515453,
   0,
   0,
   0.8630633612693448,
   0.9880545489757687,
   0.5224287125873076,
   0.7655584576672219,
   0.5587578868621627,
   0.9108491111434109,
   0.9802632497017835,
   0.9640227004704329,
   0.8488286062765426,
   0.8524379571538377,
   0.6637629012824463,
   0,
   0.9197308046554844,
   0.8740212212991818,
   0,
   0.8272047810094683,
   0.2759910921529042],
  2: [0.9536987917506862,
   0.8989509003387748,
   0.5481885916848738,
   0.37164183316101573,
   0.9253671474558207,
   0.8771320891469716,
   0.31046466994554267,
   0,
   0.8801075854199869,
   0.9871960976815842,
   0.48463693002858454,
   0.725053787291138,
   0.7103007904303162,
   0.9701708589715807,
   0.9811146491025737,
   0.9548879276595852,
   0.8630605472830313,
   0.7576597278263456,
   0.6201428731352271,
   0.848186918545766,
   0.9

In [17]:

import json
json_object = json.dumps(experiment_results, indent=4)
 
# Writing to sample.json
with open("experiment_results.json", "w") as outfile:
    outfile.write(json_object)